In [1]:
import os
import firebase_admin
from firebase_admin import credentials, db
import pandas as pd
import csv

C:\Users\lara\AppData\Local\Temp\ipykernel_10488\900411637.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# ✅ Get Firebase credentials path
cred_path = os.getenv("FIREBASE_KEY_PATH")

if not cred_path:
    raise ValueError("FIREBASE_KEY_PATH is not set. Make sure the environment variable is configured.")

# ✅ Define database URL
DATABASE_URL = "https://umsports-linksync-default-rtdb.europe-west1.firebasedatabase.app/"

# ✅ Print debug info
print("🔍 Using Firebase Key Path:", cred_path)
print("🔍 Using Database URL:", DATABASE_URL)

# ✅ Initialize Firebase if not already initialized
if not firebase_admin._apps:
    cred = credentials.Certificate(cred_path)
    firebase_admin.initialize_app(cred, {'databaseURL': DATABASE_URL})

# ✅ Check if databaseURL is set correctly
app = firebase_admin.get_app()
print("✅ Firebase initialized with options:", app.options)

# ✅ Explicitly print databaseURL
print("🔍 Confirmed databaseURL in Firebase app:", app.options.get("databaseURL"))

# ✅ Connect to Realtime Database
ref = db.reference("/")  # Root of the database

# ✅ Test writing data
ref.child("test").set({
    "message": "Realtime Database is working!"
})

print("✅ Realtime Database connection successful! Check Firebase Console for 'test' data.")

🔍 Using Firebase Key Path: C:\Users\lara\Documents\firebase_key.json
🔍 Using Database URL: https://umsports-linksync-default-rtdb.europe-west1.firebasedatabase.app/
✅ Firebase initialized with options: <firebase_admin._AppOptions object at 0x00000281004E24B0>
🔍 Confirmed databaseURL in Firebase app: https://umsports-linksync-default-rtdb.europe-west1.firebasedatabase.app/
✅ Realtime Database connection successful! Check Firebase Console for 'test' data.


In [3]:
def upload_users_to_firebase():
    file_path = "users.xlsx"
    users_df = pd.read_excel(file_path)

    users_ref = db.reference("users")  # Reference to Firebase "users" node

    for _, row in users_df.iterrows():
        username = row["Username"]
        password = row["Password"]
        name = row["Name"]

        # Add user data to Firebase
        users_ref.child(username).set({
            "password": password,
            "name": name
        })

    print("✅ All users uploaded to Firebase successfully!")

# Run the function to upload users
upload_users_to_firebase()

✅ All users uploaded to Firebase successfully!


In [6]:
def upload_sessions_to_firebase():
    file_path = "UM_SPORTS_Sessions.xlsx"  # Ensure this file is in the correct path
    sessions_df = pd.read_excel(file_path)

    # ✅ Replace NaN values with None (null in Firebase)
    sessions_df = sessions_df.where(pd.notna(sessions_df), None)

    # ✅ Convert time and timestamp columns to string format (Firebase doesn't support these objects)
    for col in ["Session_Begin", "Session_End", "Date"]:
        if col in sessions_df.columns:
            sessions_df[col] = sessions_df[col].astype(str)

    sessions_ref = db.reference("sessions")  # Reference to Firebase "sessions" node

    for index, row in sessions_df.iterrows():
        session_id = str(index)  # Unique ID for each session
        session_data = row.to_dict()

        # Add session data to Firebase
        sessions_ref.child(session_id).set(session_data)

    print("✅ All sessions uploaded to Firebase successfully!")

upload_sessions_to_firebase()

✅ All sessions uploaded to Firebase successfully!


In [5]:
def upload_formatted_sessions_to_firebase():
    file_path = "Formatted_UM_SPORTS_Sessions.xlsx"  # Ensure this file is in the correct path
    sessions_df = pd.read_excel(file_path)

    # ✅ Convert DataFrame to a List
    formatted_sessions_list = sessions_df["Formatted_Session"].dropna().tolist()

    formatted_sessions_ref = db.reference("formatted_sessions")  # Firebase node

    # ✅ Upload data as a list instead of an object
    formatted_sessions_ref.set(formatted_sessions_list)

    print("✅ All formatted sessions uploaded to Firebase successfully!")

upload_formatted_sessions_to_firebase()

✅ All formatted sessions uploaded to Firebase successfully!
